## Finding Textual Similarity

### Problem

### Context

## Importing all Required Libraries

In [36]:
import numpy as np
import pandas as pd

import re

import nltk

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk import word_tokenize
wordnet=WordNetLemmatizer()

import gensim
from gensim.models import word2vec

from sklearn.cluster import KMeans

import pickle
import sys


## Reading DataSet

In [2]:
df=pd.read_csv('D:\\python\\Text_Similarity_Dataset.csv')

In [3]:
df

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...
...,...,...,...
4018,4018,labour plans maternity pay rise maternity pay ...,no seasonal lift for house market a swathe of ...
4019,4019,high fuel costs hit us airlines two of the lar...,new media battle for bafta awards the bbc lead...
4020,4020,britons growing digitally obese gadget lover...,film star fox behind theatre bid leading actor...
4021,4021,holmes is hit by hamstring injury kelly holmes...,tsunami to hit sri lanka banks sri lanka s b...


## Shape of DataSet

In [4]:
df.shape

(4023, 3)

## Checking for null values

In [5]:
df.isnull().sum()

Unique_ID    0
text1        0
text2        0
dtype: int64

### Checking describtion for data

In [6]:
df[['text1','text2']].describe()

,text1,text2
count,4023,4023
unique,2125,2125
top,2d metal slug offers retro fun like some drill...,brown outlines third term vision gordon brown ...
freq,4,4


In [7]:
df[['text1','text2']]

,text1,text2
0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...
...,...,...
4018,labour plans maternity pay rise maternity pay ...,no seasonal lift for house market a swathe of ...
4019,high fuel costs hit us airlines two of the lar...,new media battle for bafta awards the bbc lead...
4020,britons growing digitally obese gadget lover...,film star fox behind theatre bid leading actor...
4021,holmes is hit by hamstring injury kelly holmes...,tsunami to hit sri lanka banks sri lanka s b...


## Sample of dataset from text1 and text2

In [8]:
df['text1'][0]

'savvy searchers fail to spot ads internet search engine users are an odd mix of naive and sophisticated  suggests a report into search habits.  the report by the us pew research center reveals that 87% of searchers usually find what they were looking for when using a search engine. it also shows that few can spot the difference between paid-for results and organic ones. the report reveals that 84% of net users say they regularly use google  ask jeeves  msn and yahoo when online.  almost 50% of those questioned said they would trust search engines much less  if they knew information about who paid for results was being hidden. according to figures gathered by the pew researchers the average users spends about 43 minutes per month carrying out 34 separate searches and looks at 1.9 webpages for each hunt. a significant chunk of net users  36%  carry out a search at least weekly and 29% of those asked only look every few weeks. for 44% of those questioned  the information they are looking

In [9]:
df['text2'][0]

'newcastle 2-1 bolton kieron dyer smashed home the winner to end bolton s 10-game unbeaten run.  lee bowyer put newcastle ahead when he fed stephen carr on the right flank  then sprinted into the area to power home a header from the resultant cross. wanderers hit back through stelios giannakopoulos  who ended a fluid passing move with a well-struck volley. but dyer had the last word in a game of few chances  pouncing on a loose ball after alan shearer s shot was blocked and firing into the top corner. neither side lacked urgency in the early stages of the game  with plenty of tackles flying in  but opportunities in front of goal were harder to come by. bolton keeper jussi jaaskelainen had to make two saves in quick succession midway through the first-half - keeping out shearer s low shot and dyer s close-range header - but that was the only goalmouth action of note. and it was almost out of nothing that the magpies took the lead on 35 minutes. bowyer found space with a neat turn on the

## Decode the phrases
### won't -> will not
### can't -> can not

In [10]:
def decode(phrase):
    # specific words
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general words
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [11]:
# pre_processing = []

# for sentance in (df['text1'].values):
#     sent = decode(sentance)
#     sent = sent.replace('\\r', ' ')
#     sent = sent.replace('\\"', ' ')
#     sent = sent.replace('\\n', ' ')
#     sent = re.sub('[^A-Za-z0-9]+',' ',sent)
#     sent = [wordnet.lemmatize(word) for word in sent if not word in stopwords.words('english')]
#     sent = ' '.join(sent)
#     pre_processing.append(sent.lower().strip())

## Pre-Processing of data

In [44]:
# Removing of stopwords
# Removing of Special characters
# Lemmatizing the data set  by wordnet Lemmatizer
# converting the data into lower case

In [12]:
 
preprocessed_text1 = []


for sentance in (df['text1'].values):
    sent = decode(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

In [ ]:
# updating the text1 by preprocessed_text1

In [14]:
df['text1'] = preprocessed_text1

In [15]:
df.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail spot ads internet search ...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions miss net 2025 40 uk population still ...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short ginepri fifteen year old...,ruddock backs yapp s credentials wales coach m...
3,3,diageo buy us wine firm diageo world biggest s...,mci shares climb on takeover bid shares in us ...
4,4,careful code new european directive could put ...,media gadgets get moving pocket-sized devices ...


In [32]:
df['text1'][1]

'millions miss net 2025 40 uk population still without internet access home says study around 23 million britons miss wide range essential services education medical information predicts report telecoms giant bt compares 27 million 50 uk currently online idea digital divide evaporate time wishful thinking report concludes study calls government telecoms industry come new ways lure bypassed digital revolution although percentage britons without home access fallen slightly remain digital refuseniks miss report suggests everyday tasks move online offline services become less comprehensive divide become obvious burdensome got net access predicts gap nets nots much talked predictions divide affect future generations less discussed bt set predict future patterns based current information taking account way technology changing optimists predict convergence emergence user friendly technology bridge digital divide could way mark report suggests internet access devices tends something taken alre

In [16]:
preprocessed_text2 = []

for sentance in (df['text2'].values):
    sent = decode(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
   
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

In [17]:
preprocessed_text2[:1]

['newcastle 2 1 bolton kieron dyer smashed home winner end bolton 10 game unbeaten run lee bowyer put newcastle ahead fed stephen carr right flank sprinted area power home header resultant cross wanderers hit back stelios giannakopoulos ended fluid passing move well struck volley dyer last word game chances pouncing loose ball alan shearer shot blocked firing top corner neither side lacked urgency early stages game plenty tackles flying opportunities front goal harder come bolton keeper jussi jaaskelainen make two saves quick succession midway first half keeping shearer low shot dyer close range header goalmouth action note almost nothing magpies took lead 35 minutes bowyer found space neat turn half way line striding forward picked carr right continued run perfect timing made way box met carr cross downward header far corner bolton produced little going forward point responded well level within six minutes thanks smart finish giannakopoulos jay jay okocha twisted turned edge area neat

In [ ]:
# updating the text2 with preprocessed_text2

In [18]:
df['text2'] = preprocessed_text2

In [19]:
df.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail spot ads internet search ...,newcastle 2 1 bolton kieron dyer smashed home ...
1,1,millions miss net 2025 40 uk population still ...,nasdaq planning 100m share sale owner technolo...
2,2,young debut cut short ginepri fifteen year old...,ruddock backs yapp credentials wales coach mik...
3,3,diageo buy us wine firm diageo world biggest s...,mci shares climb takeover bid shares us phone ...
4,4,careful code new european directive could put ...,media gadgets get moving pocket sized devices ...


In [45]:
# Applying lemmatizer

In [20]:
def word_tokenizer(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens
    

## Word Embeddings

In [ ]:
# Loading pre trained google news vectors after downloading the file

In [21]:
from gensim.models import Word2Vec,KeyedVectors

In [26]:
model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True)

### Comparing Similarity text between text1 and text2 using n_similarity which compares the cosine similarity between two

In [ ]:
# In this it will check whether word in text1 and text 2 present in google news vector library
# if word not present it will remove the word otherwise it will compares the simliarityb

In [ ]:
# Treating dataset as Unsupervised learning problem

In [29]:
# 0 means high similarity
# 1 low similarity

similarity=[]

for ind in df.index:
    s1=df['text1'][ind]
    s2=df['text2'][ind]
    
    if s1 == s2:
        similarity.append(0.0)
    else:
        s1words = word_tokenizer(s1)
        s2words = word_tokenizer(s2)
#  vocabulary considered in the word embeddings       
        vocab = model.vocab
        
        if len(s1words and s2words)==0:
            similarity.append(1.0)
        else:
# remove the sentence words not found in the vocab        
            for word in s1words.copy():
                if(word not in vocab):
                    
                    s1words.remove(word)
            
            for word in s2words.copy():
                if(word not in vocab):
                    s2words.remove(word)
                    
            similarity.append((1-model.n_similarity(s1words,s2words)))

## getting the unique id with similarity

In [30]:
final_score=pd.DataFrame({'Unique_ID':df.Unique_ID,
                         'Similarity_score':similarity
                         })
final_score

,Unique_ID,Similarity_score
0,0,0.389471
1,1,0.292066
2,2,0.272890
3,3,0.184955
4,4,0.171677
...,...,...
4018,4018,0.228033
4019,4019,0.403985
4020,4020,0.292103
4021,4021,0.437650


# Saving Dataframe In csv file

In [46]:
final_score.to_csv('D:\\python\\Final_Score.csv',index=False)